## Introduction

### Imports


In [5]:
import catboost as cb
from catboost import CatBoostClassifier

#depreciated imports can be cleaned up
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from tensorflow.keras import regularizers



## Data Preparation


In [7]:
# construct the file paths using os.path.join
train_path = os.path.join("input", "train.csv")
test_path = os.path.join("input", "test.csv")
sample_submission_path = os.path.join("input", "sample_submission.csv")

# load the CSV data using pandas
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_submission_data = pd.read_csv(sample_submission_path)

X = train_data
y = X.pop('prognosis')

#Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


lb = LabelBinarizer()
y_train_one_hot = lb.fit_transform(y_train)
y_test_one_hot = lb.transform(y_test)



## Exploratory Data Analysis

In [ ]:
#Completed in previous notebook

## Feature Engineering


## Catboost Model

In [37]:
y_train_one_hot_list = y_train_one_hot.tolist()
y_test_one_hot_list = y_test_one_hot.tolist()
#X_train_list = X_train.tolist()
#X_test_list = X_test.tolist()


In [63]:
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    loss_function='MultiClass',
    eval_metric='AUC',
    use_best_model = True,
    od_type = 'Iter',
    od_wait = 1000,
    random_seed=42
)

In [64]:
model.fit(X_train, y_train, 
          eval_set=(X_test, y_test), 
          cat_features=[],
          verbose=False)

In [65]:
#submission = pd.DataFrame({'prognosis': predictions})
#predictions = model.predict(test_data)

In [66]:
# Assuming you have trained the model and stored it in a variable called 'model'
probabilities = model.predict_proba(test_data)

# Get the indices of the classes sorted in descending order based on the probabilities
top_3_indices = np.argsort(-probabilities, axis=1)[:, :3]

# Get the predicted class labels for the top 3 classes
top_3_predictions = np.take(model.classes_, top_3_indices)

In [70]:
top_3_predictions = pd.DataFrame(top_3_predictions)
rect_top_labels = pd.DataFrame(top_3_predictions.apply(lambda row: ' '.join(row.astype(str)), axis=1))
output = (rect_top_labels
          .rename(columns={0: 'prognosis'})
          .set_index(test_data.id)
         )

output.to_csv('output/output.csv')#, quoting=csv.QUOTE_NONE
        


In [69]:
output

,prognosis
id,
707,Rift_Valley_fever Dengue Tungiasis
708,Dengue Chikungunya Rift_Valley_fever
709,West_Nile_fever Japanese_encephalitis Yellow_F...
710,Rift_Valley_fever Japanese_encephalitis Dengue
711,Japanese_encephalitis West_Nile_fever Zika
...,...
1005,Yellow_Fever West_Nile_fever Zika
1006,Lyme_disease Malaria West_Nile_fever
1007,Lyme_disease West_Nile_fever Rift_Valley_fever
